# TPS 12/21 - Feature Engineering

In this notebook we test out some feature engineering techniques using XGBoost with default settings to see if we get any improvement over the baseline.

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 6
TRAIN_SIZE = 500000

In [2]:
import numpy as np
import pandas as pd
import time
import os
import pyarrow
import gc

# Model/Evaluation
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, recall_score
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Load and Prepare Data

In [3]:
# Load full training data
train = pd.read_feather('../data/train.feather')

# Drop low/no variance 
train.drop(["Soil_Type7", "Id", "Soil_Type15"], axis=1, inplace=True)
train = train[train.Cover_Type != 5]

# Label Encoding
new_encoder = LabelEncoder()
train["Cover_Type"] = new_encoder.fit_transform(train["Cover_Type"])

# Split synthetic data
train, test = train_test_split(
    train, 
    train_size = TRAIN_SIZE, 
    random_state = RANDOM_SEED,
    stratify = train['Cover_Type'],
)
y_train = train['Cover_Type']


# features, data structure for summary scores
features = [x for x in train.columns if x not in ['Id','Cover_Type']]
nonsoil = [x for x in features if not x.startswith('Soil_Type')]
new_rows = list()
gc.collect()

print(f'Training Size: {train.shape[0]} rows, {train.shape[1]} cols')
print(f'Holdout Size: {test.shape[0]} rows, {test.shape[1]} cols\n')

Training Size: 500000 rows, 53 cols
Holdout Size: 3499999 rows, 53 cols



# Model

In [4]:
# XGBoost Classifier
xgb_pipeline = make_pipeline(
    XGBClassifier(
        booster = 'gbtree',
        tree_method = 'hist',
        eval_metric = 'mlogloss',
        random_state = RANDOM_SEED,
    ),
)

# Scoring Function

In [5]:
def score_features(sklearn_model, processing = None):
    
    # Original Training/Test Split
    features = [x for x in train.columns if x not in ['Id','Cover_Type']]
    X_temp, X_test = train[features], test[features]
    y_temp, y_test = train['Cover_Type'], test['Cover_Type']
    
    # Feature Engineering
    if processing:
        X_temp = processing(X_temp)
        X_test = processing(X_test)
    
    # Store the out-of-fold predictions
    test_preds = np.zeros((X_test.shape[0],6))
    oof_preds = np.zeros((X_temp.shape[0],))
    fi_scores = np.zeros((X_temp.shape[1],))
    scores, times = np.zeros(NUM_FOLDS), np.zeros(NUM_FOLDS)
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train[features],train['Cover_Type'])):
       
        # Training and Validation Sets
        X_train, X_valid = X_temp.iloc[train_idx], X_temp.iloc[valid_idx]
        y_train, y_valid = y_temp.iloc[train_idx], y_temp.iloc[valid_idx]
        
        # Create model
        start = time.time()
        model = clone(sklearn_model)
        model.fit(X_train, y_train)

        # Permutation Importance
        result = permutation_importance(
            model, X_valid, y_valid, 
            random_state=RANDOM_SEED
        )
        fi_scores += result.importances_mean / NUM_FOLDS

        # validation/holdout predictions
        valid_preds = np.ravel(model.predict(X_valid))
        oof_preds[valid_idx] = valid_preds
        test_preds += model.predict_proba(X_test)

        # Save scores and times
        scores[fold] = accuracy_score(y_valid, valid_preds)
        end = time.time()
        times[fold] = end-start
        print(f'Fold {fold} Accuracy:  {round(scores[fold], 5)} in {round(end-start,2)}s.')
        time.sleep(0.5)
    
    features = [x for x in X_temp.columns]
    nonsoil = [x for x in X_test.columns if not x.startswith('Soil_Type')]
    test_preds = np.argmax(test_preds, axis = 1)
    test_score = accuracy_score(y_test, test_preds)
    #print('\n'+model.__class__.__name__)
    print("Train Accuracy:", round(scores.mean(), 5))
    print('Test Accuracy:', round(test_score, 5))
    print(f'Training Time: {round(times.sum(), 2)}s')
    
    fi_scores = pd.Series(
        data = fi_scores, 
        index = features
    ).loc[nonsoil].sort_values()
    
    return scores.mean(), oof_preds, test_score, fi_scores

# Baseline

In [6]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline
)

new_rows.append((
    'Baseline', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95793 in 56.93s.
Fold 1 Accuracy:  0.95714 in 55.6s.
Fold 2 Accuracy:  0.95698 in 58.85s.
Fold 3 Accuracy:  0.95768 in 57.47s.
Fold 4 Accuracy:  0.95802 in 57.8s.
Fold 5 Accuracy:  0.95702 in 58.02s.
Train Accuracy: 0.95746
Test Accuracy: 0.9583
Training Time: 344.67s


Hillshade_3pm                        -0.000049
Slope                                 0.000004
Hillshade_9am                         0.000032
Aspect                                0.000083
Wilderness_Area2                      0.000098
Hillshade_Noon                        0.000413
Wilderness_Area4                      0.002781
Wilderness_Area1                      0.010013
Horizontal_Distance_To_Hydrology      0.010578
Vertical_Distance_To_Hydrology        0.018296
Wilderness_Area3                      0.019741
Horizontal_Distance_To_Fire_Points    0.031517
Horizontal_Distance_To_Roadways       0.047248
Elevation                             0.460723
dtype: float64

# Feature Engineering

1. Aspect Features
2. Hillshade Features
3. Water Features
4. Count Features
5. Water/Fire Interactions
6. Roadway Interactions
7. Elevation Interactions

## 1. Aspect Features

In [7]:
def aspect_features(data):
    df = data.copy()
    df['Aspect_360'] = df['Aspect'] % 360
    df['Aspect_Sine'] = (df['Aspect']* np.pi / 180).apply(np.sin)
    df['Aspect_Alt'] = (df['Aspect']-180).where(
        df['Aspect']+180 > 360, df['Aspect'] + 180
    )
    return df

In [8]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline, 
    aspect_features
)

new_rows.append((
    'Aspect_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95772 in 69.25s.
Fold 1 Accuracy:  0.95708 in 65.56s.
Fold 2 Accuracy:  0.9567 in 66.9s.
Fold 3 Accuracy:  0.95741 in 66.79s.
Fold 4 Accuracy:  0.95824 in 68.03s.
Fold 5 Accuracy:  0.95729 in 70.03s.
Train Accuracy: 0.95741
Test Accuracy: 0.95826
Training Time: 406.55s


Slope                                 0.000005
Aspect_360                            0.000008
Aspect_Alt                            0.000014
Hillshade_9am                         0.000042
Hillshade_3pm                         0.000044
Aspect_Sine                           0.000045
Wilderness_Area2                      0.000090
Aspect                                0.000091
Hillshade_Noon                        0.000414
Wilderness_Area4                      0.002595
Wilderness_Area1                      0.010384
Horizontal_Distance_To_Hydrology      0.010634
Vertical_Distance_To_Hydrology        0.018316
Wilderness_Area3                      0.019452
Horizontal_Distance_To_Fire_Points    0.031526
Horizontal_Distance_To_Roadways       0.047128
Elevation                             0.460853
dtype: float64

## 2. Hillshade Features

In [9]:
def hillshade_features(data):
    df = data.copy()
    shade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']

    # Clip Range
    df["Hillshade_9am_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    df["Hillshade_Noon_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    df["Hillshade_3pm_Clipped"] = df["Hillshade_9am"].clip(lower=0, upper=255)
    
    # Hillshade
    #df["Hillshade_Avg"] = df[shade_features].mean(axis=1)
    df["Hillshade_Sum"] = df[shade_features].sum(axis=1)
    df['Hillshade_Range'] = df[shade_features].max(axis=1) - df[shade_features].min(axis=1)
    
    return df

In [10]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline, 
    hillshade_features
)

new_rows.append((
    'Hillshade_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95781 in 70.08s.
Fold 1 Accuracy:  0.95735 in 69.34s.
Fold 2 Accuracy:  0.95687 in 71.8s.
Fold 3 Accuracy:  0.95758 in 70.26s.
Fold 4 Accuracy:  0.9583 in 71.35s.
Fold 5 Accuracy:  0.95672 in 71.13s.
Train Accuracy: 0.95744
Test Accuracy: 0.95832
Training Time: 423.97s


Slope                                -5.639965e-05
Hillshade_Range                      -4.000009e-05
Hillshade_3pm                        -6.400064e-06
Hillshade_3pm_Clipped                 0.000000e+00
Hillshade_Noon_Clipped                0.000000e+00
Hillshade_9am_Clipped                 4.000352e-07
Hillshade_Sum                         2.240017e-05
Hillshade_9am                         3.160000e-05
Aspect                                8.519988e-05
Wilderness_Area2                      1.095998e-04
Hillshade_Noon                        4.308001e-04
Wilderness_Area4                      2.550803e-03
Wilderness_Area1                      1.028000e-02
Horizontal_Distance_To_Hydrology      1.049960e-02
Vertical_Distance_To_Hydrology        1.824280e-02
Wilderness_Area3                      1.973800e-02
Horizontal_Distance_To_Fire_Points    3.147680e-02
Horizontal_Distance_To_Roadways       4.726600e-02
Elevation                             4.607992e-01
dtype: float64

## 3. Water Features 

In [11]:
# Helper function
def start_at_eps(series, eps=1e-10): 
    return series - series.min() + eps

def water_features(data):
    df = data.copy()
    
    # use float64 for squaring
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float64')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float64')
    pos_h_hydrology = start_at_eps(df["Horizontal_Distance_To_Hydrology"])
    pos_v_hydrology = start_at_eps(df['Vertical_Distance_To_Hydrology'])
    
    # Manhatten Distances
    df["Hydro_Taxicab"] = np.abs(df["Horizontal_Distance_To_Hydrology"]) + np.abs(df["Vertical_Distance_To_Hydrology"])
    df['Hydro_Taxicab_Pos'] = (pos_h_hydrology ** 2 + pos_v_hydrology ** 2).apply(np.sqrt).rename('Euclidean_positive_hydrology').astype(np.float32)
    
    # Euclidean Distance
    df["Hydro_Euclid"] = (df["Horizontal_Distance_To_Hydrology"]**2 + np.abs(df["Vertical_Distance_To_Hydrology"])**2)**0.5
    df['Hydro_Euclid_Pos'] = (pos_h_hydrology ** 2 + pos_v_hydrology ** 2).apply(np.sqrt)
    
    # Misc Features
    df['Water_Direction'] = df['Vertical_Distance_To_Hydrology'].apply(np.sign)
    df['Water Elevation'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    
    # Store each as float32
    df["Horizontal_Distance_To_Hydrology"] = df["Horizontal_Distance_To_Hydrology"].astype('float32')
    df["Vertical_Distance_To_Hydrology"] = df["Vertical_Distance_To_Hydrology"].astype('float32')
    df["Hydro_Taxicab"] = df["Hydro_Taxicab"].astype('float32')
    df['Hydro_Taxicab_Pos'] = df['Hydro_Taxicab_Pos'].astype('float32')
    df["Hydro_Euclid"] = df["Hydro_Euclid"].astype('float32')
    df['Hydro_Euclid_Pos'] = df['Hydro_Euclid_Pos'].astype('float32')
    df['Water_Direction'] = df['Water_Direction'].astype('float32')
    df['Water Elevation'] = df['Water Elevation'].astype('float32')
    
    return df

In [12]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline, 
    water_features
)

new_rows.append((
    'Water_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95849 in 68.5s.
Fold 1 Accuracy:  0.95758 in 64.7s.
Fold 2 Accuracy:  0.95759 in 70.47s.
Fold 3 Accuracy:  0.95856 in 66.66s.
Fold 4 Accuracy:  0.95882 in 66.24s.
Fold 5 Accuracy:  0.95808 in 70.14s.
Train Accuracy: 0.95819
Test Accuracy: 0.95905
Training Time: 406.7s


Hillshade_3pm                        -0.000075
Hillshade_9am                        -0.000068
Aspect                               -0.000013
Water_Direction                       0.000000
Hydro_Euclid_Pos                      0.000000
Slope                                 0.000022
Wilderness_Area2                      0.000057
Vertical_Distance_To_Hydrology        0.000116
Hillshade_Noon                        0.000360
Hydro_Taxicab                         0.000425
Hydro_Taxicab_Pos                     0.000612
Hydro_Euclid                          0.000806
Wilderness_Area4                      0.002606
Horizontal_Distance_To_Hydrology      0.003022
Wilderness_Area1                      0.009490
Wilderness_Area3                      0.019847
Horizontal_Distance_To_Fire_Points    0.031968
Horizontal_Distance_To_Roadways       0.047543
Water Elevation                       0.126658
Elevation                             0.326246
dtype: float64

## 4. Count Features

In [13]:
def count_features(data):
    
    df = data.copy()
    soil_features = [x for x in df.columns if x.startswith("Soil_Type")]
    wilderness_features = [x for x in df.columns if x.startswith("Wilderness_Area")]

    # Count features
    df["Soil_Count"] = df[soil_features].apply(sum, axis=1)
    df["Wilderness_Count"] = df[wilderness_features].apply(sum, axis=1)
    
    return df

In [14]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline, 
    count_features
)

new_rows.append((
    'Count_Features', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95956 in 65.3s.
Fold 1 Accuracy:  0.95948 in 67.74s.
Fold 2 Accuracy:  0.95931 in 66.42s.
Fold 3 Accuracy:  0.95981 in 67.25s.
Fold 4 Accuracy:  0.96058 in 68.31s.
Fold 5 Accuracy:  0.95864 in 68.76s.
Train Accuracy: 0.95956
Test Accuracy: 0.96026
Training Time: 403.77s


Slope                                -0.000134
Hillshade_3pm                         0.000005
Hillshade_9am                         0.000029
Aspect                                0.000073
Wilderness_Area2                      0.000098
Hillshade_Noon                        0.000474
Wilderness_Count                      0.000482
Wilderness_Area4                      0.005108
Horizontal_Distance_To_Hydrology      0.010983
Wilderness_Area1                      0.011182
Vertical_Distance_To_Hydrology        0.018890
Wilderness_Area3                      0.019424
Horizontal_Distance_To_Fire_Points    0.032689
Soil_Count                            0.036147
Horizontal_Distance_To_Roadways       0.048996
Elevation                             0.461825
dtype: float64

## 5. Water/Fire Interactions

In [15]:
def hydrofire_interactions(data):
    
    df = data.copy()
    df['Hydro_Fire_Sum'] = df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']
    df['Hydro_Fire_AbsDiff'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Fire_EpsSum'] = start_at_eps(df['Horizontal_Distance_To_Hydrology']) + start_at_eps(df['Horizontal_Distance_To_Fire_Points'])
    df['Hydro_Fire_Diff'] = df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points']
    return df

In [16]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline, 
    hydrofire_interactions
)

new_rows.append((
    'Water_Fire', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95868 in 66.33s.
Fold 1 Accuracy:  0.95716 in 68.76s.
Fold 2 Accuracy:  0.95752 in 69.79s.
Fold 3 Accuracy:  0.95812 in 71.02s.
Fold 4 Accuracy:  0.95801 in 71.4s.
Fold 5 Accuracy:  0.95752 in 69.7s.
Train Accuracy: 0.95784
Test Accuracy: 0.95844
Training Time: 417.0s


Hillshade_3pm                        -0.000013
Hydro_Fire_EpsSum                     0.000000
Slope                                 0.000081
Wilderness_Area2                      0.000099
Hillshade_9am                         0.000151
Aspect                                0.000190
Hillshade_Noon                        0.000408
Wilderness_Area4                      0.002648
Horizontal_Distance_To_Fire_Points    0.003858
Hydro_Fire_AbsDiff                    0.004232
Hydro_Fire_Diff                       0.004674
Horizontal_Distance_To_Hydrology      0.007021
Hydro_Fire_Sum                        0.008988
Wilderness_Area1                      0.010056
Vertical_Distance_To_Hydrology        0.018526
Wilderness_Area3                      0.019899
Horizontal_Distance_To_Roadways       0.047630
Elevation                             0.460796
dtype: float64

## 6. Roadway Interactions

In [17]:
def roadway_interactions(data):
    df = data.copy()
    df['Hydro_Road_1'] = abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])
    df['Hydro_Road_2'] = abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_1'] = abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])
    df['Fire_Road_2'] = abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])
    return df

In [18]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline, 
    roadway_interactions
)

new_rows.append((
    'Road_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95877 in 63.4s.
Fold 1 Accuracy:  0.95738 in 64.02s.
Fold 2 Accuracy:  0.95752 in 63.93s.
Fold 3 Accuracy:  0.9581 in 66.99s.
Fold 4 Accuracy:  0.95805 in 66.16s.
Fold 5 Accuracy:  0.95769 in 61.36s.
Train Accuracy: 0.95792
Test Accuracy: 0.9589
Training Time: 385.88s


Slope                                -0.000031
Hillshade_3pm                         0.000002
Wilderness_Area2                      0.000037
Aspect                                0.000050
Hillshade_9am                         0.000053
Hillshade_Noon                        0.000332
Hydro_Road_1                          0.002124
Wilderness_Area4                      0.002846
Hydro_Road_2                          0.003413
Fire_Road_2                           0.007075
Horizontal_Distance_To_Hydrology      0.008510
Fire_Road_1                           0.008551
Wilderness_Area1                      0.010162
Horizontal_Distance_To_Fire_Points    0.012640
Vertical_Distance_To_Hydrology        0.018387
Wilderness_Area3                      0.019986
Horizontal_Distance_To_Roadways       0.023611
Elevation                             0.460478
dtype: float64

## 7. Elevation Interactions

In [19]:
def elevation_interactions(data):
    df = data.copy()
    df['Road_Elev_Int'] = df['Horizontal_Distance_To_Roadways'] * df['Elevation']
    df['VHydro_Elev_Int'] = df['Vertical_Distance_To_Hydrology'] * df['Elevation']
    df['Elev_VHydro_Diff'] = df.Elevation - df.Vertical_Distance_To_Hydrology
    df['Elev_HHydro_Diff'] = df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2
    
    return df

In [20]:
cv_score, oof_preds, test_score, fi_scores = score_features(
    xgb_pipeline, 
    elevation_interactions
)

new_rows.append((
    'Elev_Interactions', cv_score, test_score,
     *recall_score(y_train, oof_preds, average = None)
))

fi_scores

Fold 0 Accuracy:  0.95844 in 64.26s.
Fold 1 Accuracy:  0.95795 in 65.04s.
Fold 2 Accuracy:  0.95736 in 69.35s.
Fold 3 Accuracy:  0.95891 in 72.94s.
Fold 4 Accuracy:  0.95897 in 67.39s.
Fold 5 Accuracy:  0.9576 in 67.14s.
Train Accuracy: 0.95821
Test Accuracy: 0.95901
Training Time: 406.12s


Slope                                -4.080000e-05
VHydro_Elev_Int                      -2.599987e-05
Hillshade_3pm                        -2.400058e-06
Road_Elev_Int                        -7.998064e-07
Hillshade_9am                         7.119985e-05
Aspect                                7.479999e-05
Wilderness_Area2                      7.639967e-05
Hillshade_Noon                        3.972006e-04
Vertical_Distance_To_Hydrology        4.251999e-04
Wilderness_Area4                      2.516402e-03
Horizontal_Distance_To_Hydrology      4.108000e-03
Wilderness_Area1                      9.814803e-03
Wilderness_Area3                      1.963880e-02
Horizontal_Distance_To_Fire_Points    3.189320e-02
Elev_HHydro_Diff                      3.522401e-02
Horizontal_Distance_To_Roadways       4.742160e-02
Elevation                             1.412520e-01
Elev_VHydro_Diff                      1.413984e-01
dtype: float64

# Summary

In [21]:
pd.DataFrame.from_records(
    data = new_rows,
    columns = ['features','cv_scores','holdout','recall_0', 'recall_1','recall_2','recall_3','recall_4','recall_5']
).sort_values('holdout')

,features,cv_scores,holdout,recall_0,recall_1,recall_2,recall_3,recall_4,recall_5
1,Aspect_Features,0.957408,0.958260,0.963458,0.971800,0.873815,0.191489,0.418067,0.658229
0,Baseline,0.957460,0.958297,0.963491,0.971828,0.873365,0.191489,0.417367,0.661313
2,Hillshade_Features,0.957438,0.958325,0.963731,0.971743,0.873201,0.212766,0.401961,0.660542
5,Water_Fire,0.957836,0.958441,0.963965,0.972171,0.873324,0.212766,0.413866,0.662469
6,Road_Interactions,0.957916,0.958901,0.964031,0.971647,0.876962,0.212766,0.450980,0.666838
7,Elev_Interactions,0.958208,0.959013,0.963371,0.972779,0.877330,0.212766,0.413866,0.665682
3,Water_Features,0.958188,0.959051,0.963562,0.972588,0.877739,0.234043,0.423669,0.663626
4,Count_Features,0.959562,0.960255,0.964189,0.971824,0.885505,0.170213,0.467787,0.732751
